In [1]:
%matplotlib widget

In [2]:
import os
import pickle
import copy

import numpy as np
import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats

import pyro
import pyro.infer
import pyro.infer.mcmc
import pyro.distributions as dist
import torch.distributions.constraints as constraints
from tqdm.auto import tqdm

import matplotlib.pyplot as plot
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

Let's generate data for 18 models, 1000 examples, with a, b, theta, gamma, parameters.
a, b, theta are multiple dimensions.

## Lets define prior distribution

In [3]:
n_models, n_items = 18, 1000
dimension=1
item_params_std=1

alpha_dist={}
alpha_dist = {"mu": 0.0, "std":0.3}
theta_dist = {"mu": 0.0, "std":1}

positive_transform = lambda x: torch.log(1 + torch.exp(x))


In [3]:
pyro.sample(dist.Bernoulli(0.7))

TypeError: sample() missing 1 required positional argument: 'fn'

In [4]:
def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))

# For 1D

In [5]:
# Generate betas
betas = pyro.sample("b", dist.Normal(torch.zeros(n_items), item_params_std))

# Generate gamma
log_gamma = pyro.sample("log c", dist.Normal(torch.zeros(n_items), item_params_std))
gamma = sigmoid(log_gamma)

# Generate alphas
alphas = pyro.sample(
            "a",
            dist.LogNormal(
                alpha_dist["mu"] * torch.ones(n_items),
                alpha_dist["std"],
            ),
        )

# Generate thetas
thetas = pyro.sample(
            "theta",
            dist.Normal(
                theta_dist["mu"] * torch.ones(n_models),
                theta_dist["std"],
            ),
        )

#alphas = positive_transform(alphas)
#thetas = positive_transform(thetas)
lik = dist.Bernoulli(
            gamma[None, :]
            + (1.0 - gamma[None, :])
            * sigmoid(alphas[None, :] * (thetas[:, None] - betas[None, :]))
        ).sample()
    
    
responses=lik

In [6]:
model_names = ["model_{}".format(i) for i in range(n_models)]
items_names = ["data_{}".format(i) for i in range(n_items)]

df = pd.DataFrame(data = responses.numpy().astype(int),  
                        index = model_names,  
                        columns = items_names) 
df.index.names = ['userid']
df
#df.to_csv('/Users/phumon/Documents/Research/nlu-test-sets/data_synthetic/sync_irt_all_coded.csv')

,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_990,data_991,data_992,data_993,data_994,data_995,data_996,data_997,data_998,data_999
userid,,,,,,,,,,,,,,,,,,,,,
model_0,1,1,0,1,0,1,0,0,1,1,...,1,0,0,0,1,1,1,1,1,0
model_1,1,1,1,0,1,1,1,0,1,1,...,1,1,1,1,0,1,1,0,1,0
model_2,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,0,1,1
model_3,1,1,1,1,0,1,1,1,1,1,...,1,1,1,1,0,1,1,1,0,1
model_4,1,1,1,0,1,1,0,0,0,0,...,1,1,1,1,1,0,1,1,1,0
model_5,1,1,1,0,0,1,1,0,1,0,...,1,1,0,1,1,1,1,0,1,1
model_6,1,0,1,1,1,1,1,0,1,1,...,1,1,1,1,0,0,1,1,1,1
model_7,1,0,1,0,1,1,1,1,1,1,...,1,1,1,1,1,0,1,0,1,1
model_8,1,0,1,0,1,1,0,0,1,0,...,1,1,1,0,1,0,1,1,1,1


# Multiple Dim

In [7]:
n_models, n_items = 18, 1000
dimension=6
item_params_std=1

alpha_dist={}
alpha_dist = {"mu": 0.0, "std":0.3}
theta_dist = {"mu": 0.0, "std":1}

positive_transform = lambda x: torch.log(1 + torch.exp(x))


In [8]:
# Generate betas
betas = pyro.sample("b", dist.Normal(torch.zeros(n_items, dimension), item_params_std))

# Generate gamma
log_gamma = pyro.sample("log c", dist.Normal(torch.zeros(n_items), item_params_std))
gamma = sigmoid(log_gamma)

# Generate alphas
alphas = pyro.sample(
            "a",
            dist.LogNormal(
                alpha_dist["mu"] * torch.ones(n_items, dimension),
                alpha_dist["std"],
            ),
        )

# Generate thetas
thetas = pyro.sample(
            "theta",
            dist.Normal(
                theta_dist["mu"] * torch.ones(n_models, dimension),
                theta_dist["std"],
            ),
        )

#alphas = positive_transform(alphas)
#thetas = positive_transform(thetas)
if dimension > 1:
    lik = pyro.sample(
            "likelihood",
            dist.Bernoulli(
                gamma[None, :]
                + (1.0 - gamma[None, :])
                * sigmoid(
                    torch.sum(alphas[None, :, :] * (thetas[:, None] - betas[None, :]).squeeze(), dim=-1)  
                )
            )
        )

    
responses=lik

In [9]:
betas

tensor([[-0.1421, -0.6419,  0.4618,  0.4561, -0.4018,  0.0087],
        [-0.2723, -0.5707,  0.3040, -0.3757, -0.6494,  0.5138],
        [-0.8168, -0.1598, -0.8740,  1.3224,  1.2511,  0.1567],
        ...,
        [-0.4130, -2.0918,  0.7212, -0.4905,  0.5793,  0.8294],
        [ 0.3435, -0.1647, -0.6057,  0.7487, -0.5854,  0.5503],
        [-0.1085, -0.9404, -1.5335,  1.0634, -0.7963,  0.4174]])

In [10]:
model_names = ["model_{}".format(i) for i in range(n_models)]
items_names = ["data_{}".format(i) for i in range(n_items)]

df = pd.DataFrame(data = responses.numpy().astype(int),  
                        index = model_names,  
                        columns = items_names) 
df.index.names = ['userid']
df


,data_0,data_1,data_2,data_3,data_4,data_5,data_6,data_7,data_8,data_9,...,data_990,data_991,data_992,data_993,data_994,data_995,data_996,data_997,data_998,data_999
userid,,,,,,,,,,,,,,,,,,,,,
model_0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
model_1,0,1,0,0,1,1,0,1,1,1,...,0,1,1,1,0,1,1,1,1,1
model_2,1,1,0,0,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
model_3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
model_4,1,1,1,0,0,1,1,1,1,0,...,1,1,1,1,0,1,1,0,1,1
model_5,1,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
model_6,1,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,1,1,1
model_7,1,0,0,0,1,0,1,1,1,0,...,1,1,0,1,1,1,1,1,1,1
model_8,1,0,1,0,0,1,0,1,0,0,...,1,1,1,1,1,1,1,1,1,1


In [11]:
df.to_csv('/Users/phumon/Documents/Research/nlu-test-sets/data_synthetic/dim6/sync_irt_all_coded.csv')

# Load trained params

In [12]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [13]:
def get_stats_CI(params, p=0.95, dist='normal'):
    '''
    Method to calculate lower and upper quantiles defined by `p`, mean, and variance of `param`
    
    Args:
        params: Dictionary of distribution parameters for each item keyed according to the 
                parametric distribution defined by `dist`.
        p:      Percent of distribution covered by the lower and upper interval values for each
                parameter.
        dist:   Name of parametric distribution
    
    Returns:
        return: {
            'lower': Lower interval values of each parameter,
            'upper': Upper interval values of each parameter,
            'mean' : Mean of each parameter,
            'var'  : Variance of each parameter
        }
    '''
    stats = {}
    if dist == 'normal':
        L,U = scipy.stats.norm.interval(p,loc=params['mu'], scale=torch.exp(params['logstd']))
        M,V = scipy.stats.norm.stats(loc=params['mu'], scale=torch.exp(params['logstd']))
    elif dist == 'log-normal':
        L,U = scipy.stats.lognorm.interval(p, s=torch.exp(params['logstd']), scale=torch.exp(params['mu']))
        M,V = scipy.stats.lognorm.stats(s=torch.exp(params['logstd']), scale=torch.exp(params['mu']))
    elif dist == 'beta':
        L,U = scipy.stats.beta.interval(p,a=params['alpha'], b=params['beta'])
        M,V = scipy.stats.beta.stats(a=params['alpha'], b=params['beta'])
    else:
        raise TypeError(f'Distribution type {dist} not supported.')
    
    return {
        'lower':[L],
        'upper':[U],
        'mean':[M],
        'var':[V],
    }

In [14]:
def get_plot_stats(exp_dir, alpha_dist, theta_dist, transforms, p = 0.95):
    '''
    Method to return plotting statistics for 3 parameter IRT model parameters.
    
    Args:
        exp_dir:          Path to 3 parameter IRT parameters and responses.
        alpha_dist:       Name of the item discrimination [a] distribution.
        theta_dist:       Name of the responder ability [t] distribution.
        transforms:       Dictionary of transformations to apply to each parameter type
                          where keys are parameter names and values are functions.
        p:                Percent of distribution covered by the lower and upper interval 
                          values for each parameter.
    
    Returns:
        param_plot_stats: Dictionary of parameter plot statistics where keys are parameter
                          names and values are plot statistics dictionaries as defined by
                          get_stats_CI().
    '''
    param_dists = {
        'a':alpha_dist,
        'b':'normal',
        'g':'normal',
        't':theta_dist,
    }

    dist_params = {
        'normal':['mu', 'logstd'],
        'log-normal':['mu', 'logstd'],
        'beta':['alpha', 'beta'],
    }

    pyro.clear_param_store()
    pyro.get_param_store().load(os.path.join(exp_dir, 'params.p'))

    with torch.no_grad():
        pyro_param_dict = dict(pyro.get_param_store().named_parameters())
    
    # get stats for plotting
    param_plot_stats = {}

    for param, param_dist in param_dists.items():
        temp_params = dist_params[param_dist]

        for idx, (p1_orig, p2_orig) in enumerate(zip(pyro_param_dict[f'{param} {temp_params[0]}'], pyro_param_dict[f'{param} {temp_params[1]}'])):
            p1, p2 = p1_orig.detach(), p2_orig.detach()
            
            temp_stats_df = pd.DataFrame.from_dict(
                get_stats_CI(
                    params = {
                        temp_params[0]:p1,
                        temp_params[1]:p2,
                    },
                    p=p,
                    dist = param_dist,
                )
            )
            
            temp_stats_df = temp_stats_df.applymap(transforms[param])
        
            if idx == 0:
                param_plot_stats[param] = temp_stats_df
            else:
                param_plot_stats[param] = param_plot_stats[param].append(temp_stats_df, ignore_index = True)
    
    return param_plot_stats

In [15]:
# distribution and transformation
alpha_dist = 'log-normal'
alpha_transf = 'standard'
theta_dist = 'normal'
theta_transf = 'standard'

exp_dir = os.path.join(repo, 'params_sync_d6_lr1e-3', f'alpha-lognormal-identity-dim6_theta-normal-identity_nosubsample_1.00_0.25')
p = 0.95


select_ts = {
    'standard':lambda x:x,
    'positive':lambda x:torch.log(1+torch.exp(torch.tensor(x))),
    'sigmoid':lambda x:sigmoid(torch.tensor(x)),
}

transforms = {
    'a':select_ts[alpha_transf],
    'b':select_ts['standard'],
    'g':select_ts['sigmoid'],
    't':select_ts[theta_transf],
}


param_plot_stats = get_plot_stats(
    exp_dir,
    alpha_dist,
    theta_dist,
    transforms,
    p = 0.95
)
    


In [23]:
predicted_mean_b = np.vstack(param_plot_stats['b']['mean'])
predicted_lower_b = np.vstack(param_plot_stats['b']['lower'])
predicted_upper_b = np.vstack(param_plot_stats['b']['upper'])

In [32]:
predicted_mean_b[0]

array([-0.21053958, -0.19396596, -0.22383042, -0.19341023, -0.20220777,
       -0.21051098])

In [33]:
predicted_upper_b[0]

array([1.64209507, 1.60057041, 1.58037083, 1.61118635, 1.59588726,
       1.62252275])

In [30]:
betas.numpy()

array([[-0.14211838, -0.64186525,  0.46175113,  0.4561482 , -0.40182722,
         0.00871318],
       [-0.27229014, -0.5707212 ,  0.3040263 , -0.37573102, -0.64943486,
         0.5137638 ],
       [-0.81680053, -0.1598382 , -0.87401915,  1.3224145 ,  1.2510915 ,
         0.15674032],
       ...,
       [-0.41300663, -2.0918348 ,  0.72119844, -0.4904981 ,  0.57927465,
         0.8294149 ],
       [ 0.34347433, -0.16466968, -0.60570467,  0.74874604, -0.58535683,
         0.55030984],
       [-0.10848671, -0.9404045 , -1.5335234 ,  1.0634391 , -0.7963239 ,
         0.4173716 ]], dtype=float32)

In [42]:
count=0
for row in np.greater(predicted_upper_b,betas.numpy()).astype(int):
    if 0 in row:
        print(row)
        count += 1
print(count/betas.size(0))

[1 1 1 1 1 0]
[1 1 1 0 1 1]
[1 1 1 1 0 1]
[0 1 1 1 1 1]
[1 1 0 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 0 1]
[1 1 0 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 0 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 1 0 1 1 1]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[1 1 1 1 1 0]
[0 0 1 1 1 1]
[1 1 1 0 1 1]
[1 1 1 1 1 0]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 1 0]
[1 1 1 1 1 0]
[0 1 1 1 1 1]
[1 1 0 1 1 1]
[1 1 0 1 1 1]
[0 1 1 1 1 1]
[1 1 0 1 1 1]
[1 1 1 0 1 1]
[1 0 1 1 1 1]
[0 1 1 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 1 0]
[1 1 0 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 0 1]
[1 0 1 1 1 0]
[1 1 1 1 0 1]
[1 1 0 1 1 1]
[1 1 1 0 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[0 1 1 1 1 1]
[1 1 1 1 1 0]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 0 1 1 1 1]
[1 1 1 1 1 0]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[0 1 1 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 0 1]
[1 0 1 1 1 1]
[1 1 1 1 0 1]
[1 1 1 1 1 0]
[1 1 1 0 1 1]
[1 1 1 1 0 1]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[1 1 1 1 1 0]
[1 1 1 0 1 1]
[1 1 1 0 1 1]
[0 1 1 1 1 1]
[1 1 1